In [199]:
import pandas as pd
import numpy as np
import os
import spacy
import re
import warnings
warnings.filterwarnings('ignore')


In [3]:
unique_words = pd.read_csv("../script/csv_files/unique_words_with_levels_example.csv", index_col='word')
unique_words

,counts,word_level
word,,
just,241,1.0
can,214,1.0
know,189,1.0
go,172,1.0
get,119,1.0
...,...,...
horrible,1,2.0
horizon,1,3.0
hooker,1,5.0


In [108]:
script_path = os.path.abspath("script/Drama/Black.Mirror/Black.Mirror.WEBRip.Netflix.en[cc].csv")
script_csv = pd.read_csv(script_path)

pd.set_option('display.max_columns', 100)

display(script_csv.describe())

script_csv.head(30)

,start,end,script
count,4000,4000,4000
unique,3899,4000,3715
top,00:20:03.840,00:02:03.600,Yeah.
freq,4,1,24


,start,end,script
0,00:01:56.920,00:02:03.600,"New from Wraith Babes, the hottest girls in the nastiest situations."
1,00:03:39.640,00:03:42.520,Let's play Bothergut...
2,00:04:07.720,00:04:09.680,Let's play Botherguts!
3,00:04:09.800,00:04:11.520,Yeah!
4,00:04:11.640,00:04:17.480,"Hey, come on!"
5,00:04:22.800,00:04:24.880,- Cacker-rum-bum!
6,00:04:29.080,00:04:32.560,You're hungry?
7,00:04:34.560,00:04:36.240,Come on!
8,00:04:42.160,00:04:43.920,That'll hurt!
9,00:04:45.760,00:04:47.800,"Oh, brilliant!"


### drop sentences with length < 3


In [109]:
# drop duplicates
script_csv = script_csv.drop_duplicates(subset='script')

script_csv = script_csv[~script_csv['script'].apply(lambda x: len(x.split(' ')) < 3)]
display(script_csv.describe())
script_csv.head(30)

,start,end,script
count,3156,3156,3156
unique,3089,3156,3156
top,00:15:57.640,00:02:03.600,"New from Wraith Babes, the hottest girls in the nastiest situations."
freq,3,1,1


,start,end,script
0,00:01:56.920,00:02:03.600,"New from Wraith Babes, the hottest girls in the nastiest situations."
1,00:03:39.640,00:03:42.520,Let's play Bothergut...
2,00:04:07.720,00:04:09.680,Let's play Botherguts!
4,00:04:11.640,00:04:17.480,"Hey, come on!"
10,00:05:02.080,00:05:04.200,That's fucking brilliant that.
11,00:05:37.680,00:05:41.200,That one does that sometimes. Hang on.
12,00:05:42.400,00:05:45.480,There's a little knob in here somewhere.
13,00:05:47.680,00:05:51.880,Almost the only real thing in there and even that's grown in a Petri dish.
16,00:06:42.280,00:06:44.040,"Oh, for Christ's sake!"
17,00:06:44.160,00:06:46.160,"Way to mood kill, Blubber Not!"


In [138]:
# 소문자로 시작하는 문장 제거

script_csv = script_csv[~script_csv['script'].apply(lambda x: str(x).islower())]
display(script_csv.describe())
script_csv.head(30)

,start,end,script
count,3030,3030,3030
unique,2971,3030,3030
top,00:15:57.640,00:02:03.600,"New from Wraith Babes, the hottest girls in the nastiest situations."
freq,3,1,1


,start,end,script
0,00:01:56.920,00:02:03.600,"New from Wraith Babes, the hottest girls in the nastiest situations."
1,00:03:39.640,00:03:42.520,Let's play Bothergut...
2,00:04:07.720,00:04:09.680,Let's play Botherguts!
4,00:04:11.640,00:04:17.480,"Hey, come on!"
10,00:05:02.080,00:05:04.200,That's fucking brilliant that.
11,00:05:37.680,00:05:41.200,That one does that sometimes. Hang on.
12,00:05:42.400,00:05:45.480,There's a little knob in here somewhere.
13,00:05:47.680,00:05:51.880,Almost the only real thing in there and even that's grown in a Petri dish.
16,00:06:42.280,00:06:44.040,"Oh, for Christ's sake!"
17,00:06:44.160,00:06:46.160,"Way to mood kill, Blubber Not!"


In [215]:
def spacyValidateSentence(script, nlp):
  doc = nlp(script)
  valid_sents = []
  for sent in doc.sents:
    # If at least one sentence is proper, return the valid sentence
    # ==> Skip rows that have proper nouns.
    # ==> at least one - AUX or VERB
    # ==> If first word noun - at least two - PRON or NOUN
    # ==> If first word is verb, there must be two verbs
    # ==> If verb comes before noun, order must be verb-noun-verb, not verb-noun-noun
  
    all_pos = [token.pos_ for token in doc]

    verb_in_sent = 'AUX' in all_pos or 'VERB' in all_pos
    verb_count = all_pos.count('AUX') + all_pos.count('VERB')
    first_word_is_verb = all_pos[0] == 'AUX' or all_pos[0] == 'VERB'
    is_noun = lambda x: x == 'NOUN' or x == 'PRON'
    two_nouns_in_sent = (all_pos.count('PRON') + all_pos.count('NOUN')) >= 2
    
    if 'PROPN' in all_pos:
      continue
    if is_noun(all_pos[0]) and not two_nouns_in_sent:
      continue
    if not verb_in_sent:
      continue
    if first_word_is_verb and verb_count < 2:
      continue
    # check verb noun order
    verb_nouns = list(filter(lambda pos: pos == "VERB" or is_noun(pos), all_pos))
    if len(verb_nouns) == 0:
      continue
    if verb_nouns[0] == "VERB":
      first_noun_pos = next((i for i,pos in enumerate(verb_nouns) if is_noun(pos)), -1)
      noun_count = (verb_nouns.count('PRON') + verb_nouns.count('NOUN'))
      # print(sent, "here1")
      if first_noun_pos != -1 and noun_count >= 2:
        if is_noun(verb_nouns[first_noun_pos + 1]):
          continue
    valid_sents.append(str(sent))
  return valid_sents
    

In [165]:
test = script_csv.loc[60, 'script']

nlp = spacy.load("en_core_web_sm")
spacyValidateSentence(test, nlp)

[]

In [224]:
unique_words_df = pd.DataFrame()

valid_scripts = []
valid_starts = []
valid_words = []
valid_levels = []

script_10 = script_csv.head(15)

for index, row in script_csv.iterrows():
  # if sentence contains swearwords, pass
  lowered = str(row['script']).lower()
  if "shit" in lowered or "fuck" in lowered:
    continue
  # check if all sentences have proper structure with Spacy.
  valid_sentences = spacyValidateSentence(row['script'], nlp)
  for sent in valid_sentences:
    # further sentence validation
    if (len(sent.split()) > 1 and sent[0].isupper() and sent[-1] in [".", "?", "!"] and not any(char.isdigit() for char in sent)):
      valid_starts.append(row.start)
      valid_scripts.append(sent)
      # split sent into unique words df and join with words_csv to get level column
      # get max level in level column
      # add max level value to list
      # add max level word to list
  

In [225]:

columns = {'start': valid_starts, 'script': valid_scripts} 
valid_scripts_df = pd.DataFrame(columns)

valid_scripts_df.drop_duplicates(subset='script', inplace=True)
display(valid_scripts_df.describe())

valid_scripts_df.head(30)

,start,script
count,1658,1658
unique,1566,1658
top,00:38:42.440,Let's play Bothergut...
freq,3,1


,start,script
0,00:03:39.640,Let's play Bothergut...
1,00:04:07.720,Let's play Botherguts!
2,00:04:11.640,"Hey, come on!"
3,00:05:37.680,That one does that sometimes.
4,00:05:37.680,Hang on.
5,00:05:42.400,There's a little knob in here somewhere.
6,00:06:51.640,They may as well have cut to a war crime!
7,00:07:00.840,You missed a bit.
8,00:07:09.240,Did you see that?
9,00:08:09.680,"But they started here, like you, putting their back into giving back for a brighter now."


In [149]:
def testSpacy(sentence, nlp):
  doc = nlp(sentence)
  ls = [(token, token.pos_) for token in doc]
  print("----")
  print(ls)

nlp = spacy.load("en_core_web_sm")
testSpacy("I am a cat. What about you?", nlp)

# testSpacy("The cat jumped over the dog.", nlp)
# testSpacy("He is such a dick.", nlp)
testSpacy("Can someone please help me?", nlp)
# testSpacy("There is no way you can get there today.", nlp)
# testSpacy("He did it for sure.", nlp)
# testSpacy("No one stands up for me, I have no one.", nlp)
# testSpacy("Jane stands up for me, I have no one.", nlp)

# testSpacy("Let's go to the California!", nlp)
# testSpacy("Drying my hands.", nlp)
testSpacy("Don't eat that", nlp)
testSpacy("Let's play Botherguts!", nlp)
# testSpacy("Each paying their dues like you, hoping to become a Hot Shot.", nlp)

testSpacy("Can't you just use that CBT app?", nlp)
testSpacy("Wisps you into it while you sleep.", nlp)
testSpacy("Can't you just use that CBT app?", nlp)
testSpacy("And buy what, some new shoes for my dopple to wear?", nlp)
testSpacy("I am doing this!", nlp)




----
[(I, 'PRON'), (am, 'AUX'), (a, 'DET'), (cat, 'NOUN'), (., 'PUNCT'), (What, 'PRON'), (about, 'ADP'), (you, 'PRON'), (?, 'PUNCT')]
----
[(Can, 'AUX'), (someone, 'PRON'), (please, 'INTJ'), (help, 'VERB'), (me, 'PRON'), (?, 'PUNCT')]
----
[(Do, 'AUX'), (n't, 'PART'), (eat, 'VERB'), (that, 'DET')]
----
[(Let, 'VERB'), ('s, 'PRON'), (play, 'VERB'), (Botherguts, 'NOUN'), (!, 'PUNCT')]
----
[(Ca, 'VERB'), (n't, 'PART'), (you, 'PRON'), (just, 'ADV'), (use, 'VERB'), (that, 'DET'), (CBT, 'NOUN'), (app, 'NOUN'), (?, 'PUNCT')]
----
[(Wisps, 'VERB'), (you, 'PRON'), (into, 'ADP'), (it, 'PRON'), (while, 'SCONJ'), (you, 'PRON'), (sleep, 'VERB'), (., 'PUNCT')]
----
[(Ca, 'VERB'), (n't, 'PART'), (you, 'PRON'), (just, 'ADV'), (use, 'VERB'), (that, 'DET'), (CBT, 'NOUN'), (app, 'NOUN'), (?, 'PUNCT')]
----
[(And, 'CCONJ'), (buy, 'VERB'), (what, 'PRON'), (,, 'PUNCT'), (some, 'DET'), (new, 'ADJ'), (shoes, 'NOUN'), (for, 'ADP'), (my, 'PRON'), (dopple, 'NOUN'), (to, 'PART'), (wear, 'VERB'), (?, 'PUNCT')]
--

In [75]:
def spacySentencize(text, nlp):
  print('---')
  doc = nlp(text)
  for sent in doc.sents:
    print(sent)

In [77]:
spacySentencize("I am a cat. I have two teeth.", nlp)
spacySentencize("I am a cat! I have two teeth.", nlp)
spacySentencize("Hey, look here. I have two teeth.", nlp)

---
I am a cat.
I have two teeth.
---
I am a cat!
I have two teeth.
---
Hey, look here.
I have two teeth.


In [ ]:
# at least two - PRON or NOUN
# at lease one - AUX or VERB